In [15]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
import re
import pandas as pd
import os
import time
from random import randint

In [16]:
def open_browser():
    ptions = Options()  # 調用虛擬瀏覽器Selenium
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    opt = webdriver.ChromeOptions()
    opt.add_argument('--user-agent=%s' % user_agent)
    opt.add_argument('blink-settings=imagesEnabled=False')
    opt.add_argument('--disable-dev-shm-usage')
    opt.add_argument('--disable-gpu')
    opt.add_argument('--disable-extensions')
    opt.add_argument('lang=en')
    global driver
    driver = webdriver.Chrome('./chromedriver', chrome_options=opt)
    driver.implicitly_wait(10) #隱式等待，最長等待10秒
    res = driver.get("https://www.amazon.com/")
    return res

In [ ]:
'''def Search_product_Name(product_name):
    inputElement = driver.find_element_by_class_name(
        "nav-input.nav-progressive-attribute")
    inputElement.send_keys(Keys.CONTROL, 'a')
    inputElement.send_keys(Keys.DELETE)
    inputElement.send_keys(product_name)  # 搜尋產品名
    inputElement.send_keys(Keys.ENTER)'''

In [ ]:
'''def search_urls():
    products_onPage = driver.find_elements_by_class_name(
        'a-link-normal.s-no-outline')  # 取得搜尋頁面所有商品url
    products_url_list = []
    for i in products_onPage[0:11]: #Top10 result
        tmp = i.get_attribute('href')  # webelement轉換
        products_url_list.append(tmp)
    return products_url_list'''


In [17]:
def click_user():
    global user_name_list
    user_name_list = driver.find_elements_by_class_name('a-profile')
    return user_name_list

In [18]:
def get_info():
    user_name = driver.find_elements_by_class_name('a-profile-name')
    #product_name = driver.find_elements_by_class_name('a-section.profile-at-product-title-container.profile-at-product-box-element')
    product_name = driver.find_elements_by_xpath('//*[@id="profile-at-card-container"]/div/div[2]/div[2]/div')
    profile_discriptor =  driver.find_elements_by_class_name('a-profile-descriptor')
    ratings =  driver.find_elements_by_xpath('//*[@id="profile-at-card-container"]/div/div[2]/div[2]/a/div[1]/div/i/span')
    comment = driver.find_elements_by_class_name('a-spacing-small.a-spacing-top-mini.a-color-base.profile-at-review-text.profile-at-review-text-desktop')
    asin = driver.find_elements_by_class_name('a-link-normal.profile-at-product-box-link.a-text-normal')
    #print(len(user_name),len(product_name),len(profile_discriptor),len(ratings),len(comment))
    print('Loading...')
    tmp_format = pd.DataFrame({'user_name':[],'product_name':[],'review_date_place':[],'review_rating':[],'review_content':[],'Asin':[]})
    k=0
    for i in range(len(comment)):

        if product_name[i].text != 'Product no longer available':
            try:
                _asin = str(asin[k].get_attribute('href').split('/dp/')[1].split('?ref')[0])
            except:
                _asin = str(asin[k].get_attribute('href').split('trackAsin=')[1].split('&ref')[0])
            tmp_format = tmp_format.append({'user_name':user_name[i].text,'product_name':product_name[i].text,'review_date_place':profile_discriptor[i].text,'review_rating':ratings[i].get_attribute('innerHTML'),'review_content':comment[i].text,
            'Asin':_asin},ignore_index=True)
            k+=1
        else:
            tmp_format = tmp_format.append({'user_name':user_name[i].text,'product_name':product_name[i].text,'review_date_place':profile_discriptor[i].text,'review_rating':ratings[i].get_attribute('innerHTML'),'review_content':comment[i].text,
            'Asin':"None"},ignore_index=True)
        

            
    #print(tmp_format['Asin'][0])
    return tmp_format
    #print(len(tmp_format),tmp_format)
    #print(user_name[0].text,product_name[0].text,profile_discriptor[0].text,ratings[0].get_attribute('innerHTML'),comment[0].text)

In [19]:
required_product = ['MOBIUZ']
total_format = pd.DataFrame({'user_name':[],'product_name':[],'review_date_place':[],'review_rating':[],'review_content':[],'Asin':[]})
user_format = pd.DataFrame({'user_name':[],'product_name':[],'review_date_place':[],'review_rating':[],'review_content':[],'Asin':[]}) #build empty format
review_format = pd.DataFrame({'user_name':[],'product_name':[],'review_date_place':[],'review_rating':[],'review_content':[],'Asin':[]})

In [20]:
def Crawl_model_review_user(): #抓取model的評論資料
    for p in range(1): #numbers review pages
        reviewer = driver.find_elements_by_class_name('a-profile-name')
        review_content = driver.find_elements_by_class_name('a-size-base.review-text.review-text-content')
        review_date = driver.find_elements_by_class_name('a-size-base.a-color-secondary.review-date')
        review_star = driver.find_elements_by_css_selector("i[data-hook='review-star-rating']")
        print(len( reviewer),len(review_content),len(review_date),len(review_star))
        for n in range (2,len(review_content)):
            review_format.append({'user_name':reviewer[n].text,'product_name':required_product[0],'review_date_place':review_date[n].text,'review_rating':review_star[n].get_attribute('innerHTML'),'review_content':review_content[n].text},ignore_index=True) #add data in format

        time.sleep(3)
        return review_format
        

In [21]:

def get_user_info(): #在總評論頁面點擊用戶並獲取資料
    user_format = pd.DataFrame({'user_name':[],'product_name':[],'review_date_place':[],'review_rating':[],'review_content':[],'Asin':[]}) #build empty format
    for i in range(2,len(click_user())):
        user_name_list = driver.find_elements_by_class_name('a-profile')
        print('No.',i,'User in page...')
        user_name_list[i].click()
        time.sleep(5)
        try:
            user_format = user_format.append(get_info())
            print('Userformat checking...',len(user_format),'Data collected')
            time.sleep(3)
            driver.back()
        except:
            driver.back()
            continue
        
    return user_format

執行程序

In [22]:
open_browser() #STEP1

/var/folders/9f/cbry14m53b7__fd2jstjs52h0000gn/T/ipykernel_64544/3290268548.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver', chrome_options=opt)


手動篩選適合產品頁 #STEP2

In [23]:
#STEP3 須回到商品頁
#driver.find_element_by_class_name('a-link-emphasis.a-text-bold').click() #view all review
#time.sleep(3)
for i in range(100):
    #total_format.append(Crawl_model_review_user())
    total_format = total_format.append(get_user_info())
    print('Total_format checking...\n',len(total_format))
    driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a').click()#next review page
    print('ChangePage....')
    time.sleep(3)

No. 2 User in page...
Loading...
Userformat checking... 10 Data collected
No. 3 User in page...
Loading...
Userformat checking... 14 Data collected
No. 4 User in page...
Loading...
Userformat checking... 24 Data collected
No. 5 User in page...
Loading...
Userformat checking... 34 Data collected
No. 6 User in page...
Loading...
Userformat checking... 44 Data collected
No. 7 User in page...
Loading...
Userformat checking... 54 Data collected
No. 8 User in page...
Loading...
Userformat checking... 54 Data collected
No. 9 User in page...
Loading...
Userformat checking... 64 Data collected
No. 10 User in page...
Loading...
Userformat checking... 74 Data collected
No. 11 User in page...
Loading...
Userformat checking... 84 Data collected
Total_format checking...
 84
ChangePage....
No. 2 User in page...
Loading...
Userformat checking... 10 Data collected
No. 3 User in page...
Loading...
Userformat checking... 20 Data collected
No. 4 User in page...
Loading...
Userformat checking... 30 Data co

ElementClickInterceptedException: Message: element click intercepted: Element <div class="a-profile" data-a-size="small">...</div> is not clickable at point (93, 759). Other element would receive the click: <div class="a-section cr-list-loading reviews-loading"></div>
  (Session info: chrome=97.0.4692.99)


In [24]:
print(total_format) #STEP4

        user_name                                       product_name  \
0   Brandon Lopez  MSR90 USB Swipe Magnetic Credit Card Reader 3 ...   
1   Brandon Lopez  NADAMOO Floor and Furniture Repair Kit 18 Colo...   
2   Brandon Lopez  SHOP LC DELIVERING JOY Home Table Decor Desk F...   
3   Brandon Lopez  Wireless Mouse Noise-Free 2.4G Full Size Recha...   
4   Brandon Lopez  CABLEDECONN 3M 10FT DisplayPort 1.4 Cable 8K@6...   
..            ...                                                ...   
1     West Indies  BenQ EL2870U 28 inch 4K Monitor for Gaming 1ms...   
2     West Indies  Universal Security Camera Base Mount Junction ...   
3     West Indies  Gentle Jewelry Cleaner Solution | Gold, Silver...   
4     West Indies  Bose Companion 2 Series III Multimedia Speaker...   
5     West Indies  Sunco Lighting 10 Pack 4 Inch LED Can Lights R...   

                    review_date_place        review_rating  \
0   reviewed a product · Mar 21, 2021  5 out of five stars   
1   reviewe

In [25]:
total_format.to_csv('user_review_v1_'+required_product[0]+'.csv',encoding="utf_8_sig")#STEP5